
# COVID-Team-1
### Dr. Mohanty
### Christopher Brantley
### CSC 405 01
### Last Updated: 10/18/2020
### I, Christopher Brantley, have abided by the UNCG academic policy.

In [1]:
import os 
os.chdir("../")

In [2]:
import ipynb
from ipynb.fs.full import covid_data as cd
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [38]:
# Telling pandas not convert to html tags.
pd.set_option('display.html.table_schema', True)
# Max columns and rows to display.
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 8)

### Data Paths

In [4]:
# Relative to "../"
_state_daily_cases = "./chris/DATA/state_county_daily_cases.csv"
_state_daily_deaths = "./chris/DATA/state_county_daily_deaths.csv"

_state_weekly_casees = "./chris/DATA/state_county_weekly_cases.csv"
_state_weekly_deaths = "./chris/DATA/state_county_weekly_deaths.csv"

_state_county_weekly_stats_cases = "./chris/DATA/state_county_weekly_stats_cases.csv"
_state_county_weekly_stats_deaths = "./chris/DATA/state_county_weekly_stats_deaths.csv"

# Generate weekly statistics (mean, median, mode) for number of cases and deaths across a specific state.


### Generating daily new cases for State by county.

In [5]:
def state_daily_cases_to_csv(state):
    df_primary_data = cd.get_covid_primary_data()
    df_state_cases = df_primary_data[df_primary_data.State == state].filter(regex = "County|cases")
    df_state_cases.set_index("County Name", inplace = True)
    df_daily_cases = pd.DataFrame(index = [df_state_cases.index])

    number_days = len(df_state_cases.filter(regex = "cases").columns)
    for day in range(-1, -number_days, -1):
        column_name = df_state_cases.columns[day]
        column_value = df_state_cases.iloc[:, day].sub(df_state_cases.iloc[:, day - 1]).values
        df_daily_cases.insert(0, column_name, column_value)

    first_column_name = df_state_cases.columns[0]
    first_column_values = df_state_cases.iloc[:, 0:1].values
    df_daily_cases.insert(0, first_column_name, first_column_values)
    df_daily_cases.to_csv(_state_daily_cases)

### Generating daily new deaths for State by county.

In [6]:
def state_daily_deaths_to_csv(state):
    df_primary_data = cd.get_covid_primary_data()
    df_state_deaths = df_primary_data[df_primary_data.State == state].filter(regex = "County|deaths")
    df_state_deaths.set_index("County Name", inplace = True)
    df_daily_deaths = pd.DataFrame(index = [df_state_deaths.index])

    number_days = len(df_state_deaths.filter(regex = "deaths").columns)
    for day in range(-1, - number_days, -1):
        column_name = df_state_deaths.columns[day]
        column_values = df_state_deaths.iloc[:, day].sub(df_state_deaths.iloc[:, day - 1]).values
        df_daily_deaths.insert(0, column_name, column_values)

    first_column_name = df_state_deaths.columns[0]
    first_column_values = df_state_deaths.iloc[:, 0:1].values
    df_daily_deaths.insert(0, first_column_name, first_column_values)
    df_daily_deaths.to_csv(_state_daily_deaths)

### Accessor methods for State Daily Cases and Deaths.

In [7]:
# Accessor method for state daily cases by county.
def get_state_daily_cases(_state):
    """Returns dataframe containing state daily cases by county."""
    state_daily_cases_to_csv(_state)
    return pd.read_csv(_state_daily_cases, index_col=[0])
    
# Accessor method for state daily deaths by county.
def get_state_daily_deaths(_state):
    """Returns dataframe containing state daily deaths by county."""
    state_daily_deaths_to_csv(_state)
    return pd.read_csv(_state_daily_deaths, index_col=[0])

### Generating Weekly Cases for State by County.

In [8]:
def state_weekly_cases_to_csv(state):
    df_county_cases = get_state_daily_cases(state)
    number_days = df_county_cases.filter(regex = "cases").shape[1]
    number_weeks = int(((7 - number_days % 7) + number_days) / 7)

    index_county = []
    index_weeks = []
    for county in df_county_cases.index.values:
        for week in range(0, number_weeks, 1):
            index_county.append(county)
            index_weeks.append("week " + str(week + 1))
    
    df_county_weekly = pd.DataFrame(index = [index_county, index_weeks],\
                                   columns = ["cases"])
    for county in df_county_cases.index.values:
        for column in range(0, df_county_cases.shape[1], 7):
            df_county_weekly.loc[(county, "week " + str((column // 7) + 1))] = \
                df_county_cases.loc[county].iloc[column:column + 7].sum()
    df_county_weekly.to_csv(_state_weekly_casees)

### Generating Weekly Deaths for State by County.

In [9]:
def state_weekly_deaths_to_csv(state):
    df_county_deaths = get_state_daily_deaths(state)
    number_days = df_county_deaths.filter(regex = "deaths").shape[1]
    number_weeks = int(((7 - number_days % 7) + number_days) / 7)
    
    index_county = []
    index_weeks = []
    for county in df_county_deaths.index.values:
        for week in range(0, number_weeks, 1):
            index_county.append(county)
            index_weeks.append("week " + str(week + 1))
            
    df_county_weekly = pd.DataFrame(index = [index_county, index_weeks],
                                    columns = ["deaths"])
    for county in df_county_deaths.index.values:
        for column in range(0, df_county_deaths.shape[1], 7):
            df_county_weekly.loc[(county, "week " + str((column // 7) + 1))] = \
                df_county_deaths.loc[county].iloc[column:column + 7].sum()
    df_county_weekly.to_csv(_state_weekly_deaths)

### Accessor Methods for State Weekly Cases and Deaths

In [10]:
# Accessor method for state weekly cases by county.
def get_state_weekly_cases(_state):
    state_weekly_cases_to_csv(_state)
    return pd.read_csv(_state_weekly_casees, index_col=[0, 1])
    
# Accessor method for state weekly deaths by county.
def get_state_weekly_deaths(_state):
    """Returns dataframe containing state weekly deaths by county."""
    state_weekly_deaths_to_csv(_state)
    return pd.read_csv(_state_weekly_deaths, index_col=[0, 1])

### Generating Weekly Case Stats for State by county.

In [11]:
def state_weekly_cases_stats_to_csv(_state,):
    df_primary_data = cd.get_covid_primary_data()
    df_state_pop = df_primary_data[df_primary_data.State == _state].filter(regex = "County|population")
    df_state_pop.set_index("County Name", inplace = True)
    df_weekly_normalized = get_state_weekly_cases(_state)

    # Normalizing cases to 100,000.
    for county in df_weekly_normalized.index.get_level_values(0).unique():
        pop = df_state_pop.loc[county, "population"]
        df_weekly_normalized.loc[county, :] = \
        df_weekly_normalized.loc[county, :].apply(lambda x: round(((x / pop) * 100000), 0)).values

    df_county_normalized = pd.DataFrame(index = [df_weekly_normalized.index.get_level_values(0).unique()],\
                                   columns = ["mean", "median", "mode"])
    for county in df_weekly_normalized.index.get_level_values(0).unique():
        df_county_normalized.loc[county, "mean"] = df_weekly_normalized.loc[county].mean()[0]
        df_county_normalized.loc[county, "median"] = df_weekly_normalized.loc[county].median()[0]
        df_county_normalized.loc[county, "mode"] = df_weekly_normalized.loc[county].mode().values[0,0]
    df_county_normalized.to_csv(_state_county_weekly_stats_cases)

In [29]:
def state_weekly_cases_stats_to_csv(state):
    df_primary_data = cd.get_covid_primary_data()
    df_state_pop = df_primary_data[df_primary_data.State == state].filter(regex = "County|population")
    df_state_pop.set_index("County Name", inplace = True)
    df_weekly_normalized = get_state_weekly_cases(state)

    # Normalizing cases to 100,000.
    for county in df_weekly_normalized.index.get_level_values(0).unique():
        pop = df_state_pop.loc[county, "population"]
        df_weekly_normalized.loc[county, :] = \
        df_weekly_normalized.loc[county, :].apply(lambda x: round(((x / pop) * 100000), 0)).values

    df_county_normalized = pd.DataFrame(index = [df_weekly_normalized.index.get_level_values(0).unique()],\
                                   columns = ["mean", "median", "mode"])
    for county in df_weekly_normalized.index.get_level_values(0).unique():
        df_county_normalized.loc[county, "mean"] = df_weekly_normalized.loc[county].mean()[0]
        df_county_normalized.loc[county, "median"] = df_weekly_normalized.loc[county].median()[0]
        df_county_normalized.loc[county, "mode"] = df_weekly_normalized.loc[county].mode().values[0,0]
    df_county_normalized.to_csv(_state_county_weekly_stats_cases)

### Generating Weekly Deaths Stats for State by county.

In [31]:
def state_weekly_deaths_stats_to_csv(_state,):
    df_primary_data = cd.get_covid_primary_data()
    df_state_pop = df_primary_data[df_primary_data.State == _state].filter(regex = "County|population")
    df_state_pop.set_index("County Name", inplace = True)
    df_weekly_normalized = get_state_weekly_deaths(_state)

    # Normalizing cases to 100,000.
    for county in df_weekly_normalized.index.get_level_values(0).unique():
        pop = df_state_pop.loc[county, "population"]
        df_weekly_normalized.loc[county, :] = \
        df_weekly_normalized.loc[county, :].apply(lambda x: round(((x / pop) * 100000), 0)).values

    df_county_normalized = pd.DataFrame(index = [df_weekly_normalized.index.get_level_values(0).unique()],\
                                   columns = ["mean", "median", "mode"])
    for county in df_weekly_normalized.index.get_level_values(0).unique():
        df_county_normalized.loc[county, "mean"] = round(df_weekly_normalized.loc[county].mean()[0], 0)
        df_county_normalized.loc[county, "median"] = round(df_weekly_normalized.loc[county].median()[0])
        df_county_normalized.loc[county, "mode"] = round(df_weekly_normalized.loc[county].mode().values[0,0], 0)
    df_county_normalized.to_csv(_state_county_weekly_stats_deaths)

### Accessor Methods for State Weekly Cases and Deaths

In [35]:
# Accessor method for state weekly cases by county.
def get_county_weekly_cases_stats(_state):
    state_weekly_cases_stats_to_csv(_state)
    return pd.read_csv(_state_county_weekly_stats_cases, index_col=[0])
    
# Accessor method for state weekly deaths by county.
def get_county_weekly_deaths_stats(_state):
    """Returns dataframe containing state weekly deaths by county."""
    state_weekly_deaths_stats_to_csv(_state)
    return pd.read_csv(_state_county_weekly_stats_deaths, index_col=[0])

# Task 1.1 Generate weekly statistics (mean, median, mode) for number of cases and deaths across a specific state. 

In [39]:
df_fl_county_stats_cases = get_county_weekly_cases_stats("FL")
display(df_fl_county_stats_cases)

,mean,median,mode
Alachua County,72.400000,19.0,0.0
Baker County,132.371429,14.0,0.0
Bay County,93.714286,7.0,0.0
Bradford County,97.600000,14.0,0.0
...,...,...,...
Volusia County,53.142857,14.0,0.0
Wakulla County,88.542857,15.0,0.0
Walton County,71.571429,20.0,0.0
Washington County,117.971429,16.0,0.0


In [61]:
for data_type, dataset in zip(("cases", "deaths"),\
                              (get_county_weekly_cases_stats("FL"), get_county_weekly_deaths_stats("FL"))):
    weekly_mean = dataset["mean"].mean()
    weekly_median = dataset["median"].median()
    weekly_mode = dataset["mode"].mode()
    print("Weekly mean for Florida {} is:   {:.2f}".format(data_type, weekly_mean))
    print("Weekly median for Florida {} is: {:d}".format(data_type, int(weekly_median)))
    print("Weekly mode for Florida {} is:   {:d}".format(data_type, int(weekly_mode[0])))
    print()

Weekly mean for Florida cases is:   98.77
Weekly median for Florida cases is: 18
Weekly mode for Florida cases is:   0

Weekly mean for Florida deaths is:   1.66
Weekly median for Florida deaths is: 0
Weekly mode for Florida deaths is:   0



# Task 1.2 Compare the data against other states.

# Task 1.3 Identify counties with high case and death rates.

# Task 1.4 Plot daily trends (cases and deaths, new cases) of state and top 5 infected counties. Utilize aggregrate, normalized by population, and log normalized values.